# competition 2 scatch notebooks

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [9]:
df = pd.read_excel('../default_of_credit_card_clients.xls', header=0)

df.columns = df.iloc[0,:]

df.rename(index=str, columns={"default payment next month": "Y"}, inplace=True)

df.drop('ID', inplace=True, axis=0)

y = df.Y

df.head()
df.dtypes

ID
LIMIT_BAL    object
SEX          object
EDUCATION    object
MARRIAGE     object
AGE          object
PAY_0        object
PAY_2        object
PAY_3        object
PAY_4        object
PAY_5        object
PAY_6        object
BILL_AMT1    object
BILL_AMT2    object
BILL_AMT3    object
BILL_AMT4    object
BILL_AMT5    object
BILL_AMT6    object
PAY_AMT1     object
PAY_AMT2     object
PAY_AMT3     object
PAY_AMT4     object
PAY_AMT5     object
PAY_AMT6     object
Y            object
dtype: object

In [3]:
y.value_counts

<bound method IndexOpsMixin.value_counts of 1        1
2        1
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       1
15       0
16       0
17       1
18       0
19       0
20       0
21       0
22       1
23       1
24       1
25       0
26       0
27       1
28       0
29       0
30       0
        ..
29971    0
29972    0
29973    0
29974    1
29975    1
29976    0
29977    1
29978    0
29979    0
29980    0
29981    0
29982    0
29983    1
29984    0
29985    0
29986    0
29987    0
29988    0
29989    0
29990    0
29991    0
29992    1
29993    0
29994    0
29995    1
29996    0
29997    0
29998    1
29999    1
30000    1
Name: Y, Length: 30000, dtype: object>

From the UCI dataset website:

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables: 
- X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit. 
- X2: Gender (1 = male; 2 = female). 
- X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
- X4: Marital status (1 = married; 2 = single; 3 = others). 
- X5: Age (year). 
- X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6 = the repayment status in September, 2005;
- X7 = the repayment status in August, 2005; . . .;X11 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
- X12-X17: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005. 
- X18-X23: Amount of previous payment (NT dollar). X18 = amount paid in September, 2005; X19 = amount paid in August, 2005; . . .;X23 = amount paid in April, 2005. 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(df, y, random_state=42)

In [6]:
clf = RandomForestClassifier()

clf.fit(X_train, y_train)

ValueError: Unknown label type: 'unknown'